#### Preparar Ambiente

In [1]:
import sys
!conda install -c conda-forge --yes --prefix {sys.prefix} python-snappy fastparquet snappy

Solving environment: failed with initial frozen solve. Retrying with flexible solve.
Solving environment: done


==> WARNING: A newer version of conda exists. <==
  current version: 4.8.4
  latest version: 4.10.3

Please update conda by running

    $ conda update -n base -c defaults conda



## Package Plan ##

  environment location: /home/ec2-user/anaconda3/envs/python3

  added / updated specs:
    - fastparquet
    - python-snappy
    - snappy


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    astroid-2.5                |   py36h5fab9bb_1         297 KB  conda-forge
    flask-cors-3.0.8           |             py_0          14 KB  conda-forge
    fsspec-0.8.7               |     pyhd8ed1ab_0          72 KB  conda-forge
    lxml-4.6.2                 |   py36h04a5ba7_1         1.5 MB  conda-forge
    pillow-8.1.0               |   py36ha6010c0_2         670 KB  conda-forge
    pip-21.

In [2]:
import boto3
import io
import pandas as pd

#### Seleccionar paises

In [3]:
paises = ['BO','EC','CL','VE','AR','CR','DO','SV','GT','HN','NI','PA','PY','PE','UY','BZ','GY','JM','TT','CO','MX','BR']
len(paises)

22

In [4]:
pais = 'LATAM'

#### Creando funciones

In [5]:
# Read single parquet file from S3
def pd_read_s3_parquet(key, bucket, s3_client=None, **args):
    if s3_client is None:
        s3_client = boto3.client('s3')
    obj = s3_client.get_object(Bucket=bucket, Key=key)
    return pd.read_parquet(io.BytesIO(obj['Body'].read()), **args)

# Read multiple parquets from a folder on S3 generated by spark
def pd_read_s3_multiple_parquets(filepath, bucket, s3=None, 
                                 s3_client=None, verbose=False, **args):
    if not filepath.endswith('/'):
        filepath = filepath + '/'  # Add '/' to the end
    if s3_client is None:
        s3_client = boto3.client('s3')
    if s3 is None:
        s3 = boto3.resource('s3')
    
    s3_keys = [item.key for item in s3.Bucket(bucket).objects.filter(Prefix=filepath)]
    if not s3_keys:
        print('No parquet found in', bucket, filepath)
    elif verbose:
        print('Load parquets:')
        for p in s3_keys: 
            print(p)
    dfs = [pd_read_s3_parquet(key, bucket=bucket, s3_client=s3_client, **args) 
           for key in s3_keys]
    return pd.concat(dfs, ignore_index=True)

#### Descargando datos

In [6]:
tabla_matrizOD_cities_pais_final = f'historico_mod_cities_{pais}_final'
tabla_mod = f'graphdata-mobility-public/mod/{pais}/{tabla_matrizOD_cities_pais_final}'
bucket = f'iadbprod-csd-hub-analyticaldata'

In [7]:
df = pd_read_s3_multiple_parquets(tabla_mod, bucket, s3_client = boto3.client('s3'))

In [8]:
df.head(10)

,year,month,day,id_city,id_cityh,n
0,2020,3,11,345.0,288.0,5
1,2020,3,11,78.0,154.0,5
2,2020,3,11,670.0,660.0,5
3,2020,3,11,1134.0,1145.0,5
4,2020,3,11,514.0,621.0,5
5,2020,3,11,188.0,202.0,5
6,2020,3,11,1387.0,1368.0,5
7,2020,3,11,670.0,683.0,5
8,2020,3,11,926.0,1063.0,5
9,2020,3,11,1449.0,1438.0,5


In [9]:
df.to_csv('historico_mod_cities_latam.zip', compression = 'zip')

In [10]:
df.to_stata('historico_mod_cities_latam.dta')

In [11]:
df2 = pd.read_csv('historico_mod_cities_latam.zip')

In [12]:
df2

,Unnamed: 0,year,month,day,id_city,id_cityh,n
0,0,2020,3,11,345.0,288.0,5
1,1,2020,3,11,78.0,154.0,5
2,2,2020,3,11,670.0,660.0,5
3,3,2020,3,11,1134.0,1145.0,5
4,4,2020,3,11,514.0,621.0,5
...,...,...,...,...,...,...,...
1936204,1936204,2020,3,21,19.0,52.0,2
1936205,1936205,2020,3,21,290.0,244.0,2
1936206,1936206,2020,3,21,1171.0,1281.0,2
1936207,1936207,2020,3,21,1411.0,1403.0,2
